#### 1. Import pandas library

In [14]:
import pandas as pd

#### 2. Import users table:

In [15]:
users = pd.read_csv('https://raw.githubusercontent.com/renatorbf/lab-data-cleaning/master/your-code/users_table.csv')

#### 3. Rename Id column to userId

In [16]:
users.rename(columns={'Id':'userId'}, inplace=True)

#### 4. Import posts table:

In [17]:
posts = pd.read_csv('https://raw.githubusercontent.com/renatorbf/lab-data-cleaning/master/your-code/posts_table.csv')

#### 5. Rename Id column to postId and OwnerUserId to userId

In [24]:
posts.rename(columns={'Id':'postId'}, inplace=True)
posts.rename(columns={'OwnerUserId':'userId'}, inplace=True)

#### 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

In [25]:
new_users = users[['userId', 'Reputation', 'Views', 'UpVotes', 'DownVotes']]

new_posts = posts[['postId', 'Score', 'userId', 'ViewCount', 'CommentCount']]

#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [28]:
merged = pd.merge(left= new_users,
         right= new_posts,
         how = 'outer',
         left_on = 'userId',
         right_on = 'userId')

#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [30]:
merged.isnull().value_counts()

userId  Reputation  Views  UpVotes  DownVotes  postId  Score  ViewCount  CommentCount
False   False       False  False    False      True    True   True       True            32187
                                               False   False  True       False           23572
                                                              False      False           15390
True    True        True   True     True       False   False  True       False             533
                                                              False      False             505
dtype: int64

#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 